In [19]:
import pandas as pd
import re

# Sample Series
data = {'text': ['Hello (World)', 'Python (Programming)', 'Data Science']}
df = pd.DataFrame(data)

# List of strings to search for
search_strings = ['(World)', 'Python']

# Create a boolean mask indicating if any string in the Series contains the search strings
mask = df['text'].str.contains('|'.join(map(re.escape, search_strings)))

# Use the mask to filter the rows with matching strings
result = df[mask]

# Get the indices of the matching rows
indices = result.index

print(indices)

Int64Index([0, 1], dtype='int64')


In [20]:
mask

0     True
1     True
2    False
Name: text, dtype: bool